# Bayesian hierarchical modelling for extreme value distributions

This notebook tests whether it's possible to build a Bayesian Hierarchical Model (BHM) to fit extreme value distributions to daily maximum wind gust data from a collection of automatic weather stations. 

Hierarchical models are useful where there are multiple datasets that have parameters from a shared distribution. That is, the parameters of GPD fits of observations at each station are drawn from a common distribution, which we also need to determine (and use a Bayesian approach to do that too). 

## Generalised Pareto Distribution

Parameters of the GPD:


## Data

Daily maximum wind speeds for a set of automatic weather stations (AWS) in South East Queensland.

## Validation

Comparison to Matt Mason's estimation used in SWHA SEQ

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import pymc3 as pm
from scipy.stats import genpareto

import matplotlib.pyplot as plt


Read in all daily maximum wind gust data from stations in the region

In [ ]:
data = pd.read_pickle()

station_ids = data.stnId.unique()
bins = np.arange(60, 200, 2)


Non-hierarchical model:

In [ ]:
traces = {}
bins = np.arange(60, 200, 2)

for stn in station_ids:
    datafile = f(stn)
    sdata = pd.read_pickle(datafile)
    sws = sdata.windgust.values
    shist, bin_edges = np.histogram(sws, bins, density=True)

    with pm.Model() as stnModel:
        mu = pm.Normal('mu', mu=0, sigma=1)
        sigma = pm.Normal('sigma', mu=0, sigma=1)
        xi = pm.HalfCauchy('xi', beta=1)
        eps = pm.HalfCauchy('eps', beta=1)
        fit = genpareto.pdf(bins, xi, mu, sigma) # Some function to fit the distribution
        y_like = pm.Normal('y_like', mu=fit, sigma=eps, observed=shist)
        trace = pm.sample(progressbar=False)

    traces[stn] = trace

Hierarchical model:

In [ ]:
with pm.Model() as hmodel:
    # Hyperpriors
    mu_mu = pm.Normal("mu_mu", mu=0., sigma=10)
    sigma_mu = pm.HalfCauchy('sigma_mu', beta=5
                             
                             )
    #mu_sigma = pm.Normal("mu_sigma", mu=0., sigma=1)
    #sigma_sigma = pm.HalfCauchy('sigma_sigma', beta=1)
    #beta_xi = pm.HalfCauchy("beta_xi", beta=1)
    # Station-level parameters
    mu = pm.Normal('mu', mu=mu_mu, sigma=sigma_mu, shape=len(station_ids))
    sigma = pm.HalfCauchy('sigma', beta=5, shape=len(station_ids))
    xi = pm.Exponential('xi', lam=1, shape=len(station_ids))

    eps = pm.HalfCauchy('eps', beta=1)

    for i, stnidx in enumerate(station_ids):
        stationdata = data[data['station']==stnidx]['windgust']
        
        @pm.potential
        def gpd_potential(value=stationdata, mu=mu[i], sigma=sigma[i], xi=xi[i]):
            return -np.sum(np.log(genpareto.pdf(value, xi, loc=mu, scale=sigma)))
        
    obs = pm.DensityDist("obs", gpd_potential, observed=stationdata)


In [ ]:
with hmodel:
    hierarchical_trace = pm.sample()

In [ ]:
pm.traceplot(hierarchical_trace)